## 1. Importar datos desde PDF 

### 1.1 Librerías

In [9]:
#pip install camelot-py[cv] tabula-py

In [1]:
import requests
import os
import json
import pandas as pd
import urllib.request, json, csv
import numpy as np
from tqdm import tqdm_notebook as tqdm
import tabula
import camelot

### 1.2 Importar datos de PDF

In [141]:
dbanco = tabula.read_pdf("https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf", pages="all", stream=True)

In [142]:
# Observamos que hay un error al importar la página 2
print([len(i.columns) for i in dbanco])

[5, 6, 5, 5, 5, 5, 5, 5]


In [143]:
print(len(dbanco))
dbanco[0]

8


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,OFICINA,DIRECCION,DISTRITO,PROVINCIA,DEPARTAMENTO
1,BAGUA GRANDE,AV. CHACHAPOYAS 2044,BAGUA GRANDE,UTCUBAMBA,AMAZONAS
2,CHIMBOTE,AV. FRANCISCO BOLOGNESI 696 - CHIMBOTE,CHIMBOTE,SANTA,ANCASH
3,HUARAZ,CALLE 2 ESQ. JR. JOSE SUCRE - PLAZA DE ARMAS,HUARAZ,HUARAZ,ANCASH
4,MEGAPLAZA CHIMBOTE,"AV. VICTOR R. HAYA DE LA TORRE 4694, LC B4 Y B...",CHIMBOTE,SANTA,ANCASH
5,NaN,GRAN CHAVIN,NaN,NaN,NaN
6,MERCADO MODELO CHIMBOTE,"JR. ELIAS AGUIRRE 498, ESQ. CON JR. LADISLAO E...",CHIMBOTE,SANTA,ANCASH
7,NUEVO CHIMBOTE,"CA. PROGR. VIVIENDA ETAPA I, UNIDAD U-1, NÚCLE...",NUEVO CHIMBOTE,SANTA,ANCASH
8,NaN,"BUENOS AIRES, MZ. F LT. 1",NaN,NaN,NaN
9,ANDAHUAYLAS,AV. PERU 342,ANDAHUAYLAS,ANDAHUAYLAS,APURIMAC


In [144]:
# Se elimina columna vacía
dbanco[1].dropna(how='all', axis=1, inplace=True)

In [145]:
for i in range(len(dbanco)):
    dbanco[i] = dbanco[i].reset_index().T.reset_index().T.reset_index()
    dbanco[i].drop(dbanco[i].iloc[:, 0:2], axis = 1, inplace=True)
    dbanco[i].set_axis(["oficina", "direccion", "distrito", "provincia", "departamento"], axis=1, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_17432\356026288.py:4: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  dbanco[i].set_axis(["oficina", "direccion", "distrito", "provincia", "departamento"], axis=1, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17432\356026288.py:4: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  dbanco[i].set_axis(["oficina", "direccion", "distrito", "provincia", "departamento"], axis=1, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17432\356026288.py:4: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  dbanco[i].set_axis(["oficina", "direccion", "distrito", "provincia", "departamento"], axis=1, inplace=True)
C:\U

In [146]:
# En la data de la primera hoja se elimina las primeras dos columnas mal importadas
dbanco[0]= dbanco[0].iloc[2:, :]

In [147]:
# Se concatena toda la data
bbva_peru = pd.concat(dbanco, ignore_index= True)

In [152]:
# Nos quedamos solo con Lima
bbva_peru["departamento"].replace(["LIMA.1"], ["LIMA"], inplace =   True)
print(bbva_peru['departamento'].value_counts())
bbva_lima = bbva_peru.loc[bbva_peru["departamento"] == "LIMA", :]

LIMA             206
AREQUIPA          12
LA LIBERTAD       12
PIURA             10
JUNIN              9
LAMBAYEQUE         7
SAN MARTIN         6
CUSCO              6
ICA                5
ANCASH             5
LORETO             4
CAJAMARCA          4
PUNO               3
UCAYALI            3
HUANUCO            3
MOQUEGUA           2
APURIMAC           2
TACNA              2
MADRE DE DIOS      1
PASCO.1            1
AYACUCHO           1
TUMBES             1
AMAZONAS           1
Name: departamento, dtype: int64
